# 8 - Memory Management
##  GADK - The Simple Way: Using output_key (for Agent Text Replies) 
- Using Google Agent Development Kit this is the easiest method if you just want to save your agent's final text response directly into the state. When you set up your LlmAgent, just tell it the output_key you want to use. The Runner sees this and automatically creates the necessary actions to save the response to the state when it appends the event.
- Let's look at a code example demonstrating state update via output_key 

In [1]:
# Import necessary classes from the Google Agent Developer Kit (ADK)
# LlmAgent: Represents an agent powered by a Large Language Model.
# InMemorySessionService: A service for managing sessions in memory (useful for testing).
# Session: Represents a single conversation thread or interaction session.
# Runner: Orchestrates the execution of agents within a session.
# Content, Part: Classes used to represent the content of messages and events.
from google.adk.agents import LlmAgent
from google.adk.sessions import InMemorySessionService, Session
from google.adk.runners import Runner
from google.genai.types import Content, Part

# Define an LlmAgent with an output_key.
# The output_key specifies that the agent's final text response
# should be automatically saved into the session's state under this key.
greeting_agent = LlmAgent(
    name="Greeter",  # Assign a name to the agent
    model="gemini-2.0-flash",  # Specify the language model to use (conceptual model name here)
    instruction="Generate a short, friendly greeting.",  # Provide instructions to the agent
    output_key="last_greeting",  # <-- The agent's final text output will be saved to session.state['last_greeting']
)

In [2]:
# --- Setup Runner and Session ---
# Define parameters for the application, user, and session.
app_name, user_id, session_id = "state_app", "user1", "session1"

# Initialize the InMemorySessionService to manage sessions.
session_service = InMemorySessionService()

# Initialize the Runner, linking it to the agent and session service.
# The Runner is responsible for processing user input, running the agent,
# and updating the session.
runner = Runner(
    agent=greeting_agent,  # The agent to run
    app_name=app_name,  # The application name
    session_service=session_service,  # The session service to use
)

# Create a new session for the interaction using the session service.
# You can optionally provide initial state data here.
session = session_service.create_session(
    app_name=app_name, user_id=user_id, session_id=session_id
)

# Print the initial state of the session (should be empty or contain initial data).
print(f"Initial state: {session.state}")

Initial state: {}


In [3]:
# --- Run the Agent ---
# Define the user's message content.
user_message = Content(parts=[Part(text="Hello, how are you?")])

# Run the runner with the user's message.
# The runner handles the interaction flow:
# 1. Appends the user's message to the session events.
# 2. Calls the agent with the session context.
# 3. When the agent produces a final response, the Runner
#    automatically uses the 'output_key' to create a state_delta
#    and appends the agent's response event with this state_delta.
# 4. The SessionService (when appending the event) applies the state_delta
#    to the session's state.
print("\n--- Running the agent ---")
for event in runner.run(
    user_id=user_id, session_id=session_id, new_message=user_message
):
    # Iterate through the events yielded by the runner (user input, agent responses, tool calls/results).
    # In this simple case, we just check for the final response.
    if event.is_final_response():
        # This block is executed when the agent's final response event is yielded.
        # At this point, the Runner has already processed the output_key
        # and the SessionService has updated the session state.
        print(
            f"Agent responded."
        )  # Indicate that the agent's final response was received
        # The actual response text is in event.content, but the output_key
        # mechanism means it's also now in the session state.


--- Running the agent ---
Agent responded.


In [4]:
# --- Check Updated State ---
# After the runner completes (or the loop finishes processing the final event),
# retrieve the session again to see the updated state.
# updated_session = session_service.get_session(app_name, user_id, session_id)
# get_session(*, app_name: str, user_id: str, session_id: str,
# config: GetSessionConfig | None = None) -> Session

# updated_session = session_service.get_session(app_name, user_id, session_id)
updated_session = session_service.get_session(
    app_name=app_name, user_id=user_id, session_id=session_id, config=None
)


# Print the state after the agent run.
# You should see the agent's final response saved under the key defined by output_key.
print(f"\nState after agent run: {updated_session.state}")

# Expected output might include: {'last_greeting': 'Hello there! How can I help you today?'}
# The exact greeting text depends on the LLM's response based on the instruction.

# Clean up the session (optional, but good practice in longer-running apps)
session_service.delete_session(
    app_name=app_name, user_id=user_id, session_id=session_id
)


State after agent run: {'last_greeting': 'Hello there! How can I help you today?\n'}
